<a href="https://colab.research.google.com/github/Tatianabeg/-/blob/main/%D0%B2%D0%B5%D0%B1_%D0%BF%D1%80%D0%B8%D0%BB%D0%BE%D0%B6%D0%B5%D0%BD%D0%B8%D0%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Подготовка данных :
Для разработки прост0й веб-платформы, которая позволяет вводить данные пользователей и предсказывать уровень удовлетворенности на основе моделей машинного обучения, можно использовать Flask в сочетании с Google Colab для быстрой разработки и внедрения.

Модели обучения : Создаем и обучаем модель на основе данных.
Создание веб-интерфейса :
используйте Flask для создания веб-интерфейса, куда пользователи могут вводить данные.


 Эта команда используется для аутентификации  аккаунта на платформе Ngrok. Ngrok — это инструмент, который позволяет создавать безопасные туннели к локальным серверам через интернет. Это полезно для разработки веб-приложений, когда  нужно предоставить доступ к  локальному серверу (например, Flask или другой веб-сервер) извне.
-  Заменить YOUR_AUTH_TOKEN на  фактический токен аутентификации, который вы можете получить после регистрации на сайте Ngrok. Этот токен связывает локальную установку Ngrok с вашим аккаунтом и позволяет использовать нужные функции.

In [ ]:
!ngrok authtoken YOUR_AUTH_TOKEN


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


Следующая команда устанавливает несколько библиотек Python с помощью менеджера пакетов pip. Каждая из этих библиотек имеет свои функции и предназначение:
  - pandas: Библиотека для манипуляции и анализа данных. Она предоставляет структуры данных и функции для работы с табличными данными (DataFrame).
  - numpy: Библиотека для работы с многомерными массивами и матрицами, а также для выполнения математических операций над ними.
  - scikit-learn: Библиотека для машинного обучения в Python. Она включает в себя множество алгоритмов и инструментов для обработки данных, обучения моделей и оценки их качества.
  - Flask: Легковесный веб-фреймворк для создания веб-приложений на Python. Он позволяет легко разрабатывать веб-приложения.
  - joblib: Библиотека для сериализации объектов Python (сохранения объектов на диск) и параллельных вычислений. Часто используется для сохранения обученных моделей машинного обучения.
  - pyngrok: Обертка вокруг Ngrok, которая упрощает использование этого инструмента в Python-приложениях.

In [ ]:
!pip install pandas numpy scikit-learn Flask joblib pyngrok


Вместе эти команды позволяют  настроить среду разработки для работы с данными, машинным обучением и веб-приложениями. Сначала  аутентифицируется личный аккаунт Ngrok, а затем устанавливаются необходимые библиотеки для анализа данных, построения моделей машинного обучения и создания веб-приложений.

### Обучение модели


Обучение модели происходит следующим образом:

1. **Подготовка данных**: Данные загружаются и обрабатываются (включая кодирование категориальных переменных).
2. **Разделение на обучающую и тестовую выборки**: Данные делятся на две части — одна часть используется для обучения модели (обучающая выборка), а другая — для проверки ее качества (тестовая выборка).
3. **Предобработка**: Числовые признаки нормализуются с помощью стандартизации перед подачей в модель.
4. **Настройка гиперпараметров**: Используется метод Grid Search для нахождения оптимальных значений гиперпараметров модели через перекрестную проверку.
5. **Обучение модели**: После нахождения лучших параметров модель обучается на обучающей выборке.
6. **Оценка качества**: Модель тестируется на тестовой выборке с использованием метрики RMSE.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
import joblib
from pyngrok import ngrok
from flask import Flask, request, render_template_string

np.random.seed(42)
data = {
    'age': np.random.randint(18, 70, size=100),
    'gender': np.random.choice(['Male', 'Female'], size=100),
    'health': np.random.randint(1, 6, size=100),
    'satisfaction': np.random.randint(1, 6, size=100),
    'stress_level': np.random.randint(1, 6, size=100),
    'income': np.random.randint(20000, 100000, size=100),
    'exercise_frequency': np.random.randint(0, 7, size=100),
    'sleep_hours': np.random.uniform(4, 10, size=100),
    'diet_quality': np.random.randint(1, 6, size=100)
}

df = pd.DataFrame(data)
df = pd.get_dummies(df, columns=['gender'], drop_first=True)

# Определение целевой переменной и признаков
target = 'satisfaction'
features = df.drop(columns=[target])

X_train, X_test, y_train, y_test = train_test_split(features, df[target], test_size=0.2, random_state=42)

# Определение числовых признаков
numeric_features = features.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Создание предобработчика признаков
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features)
    ])

# Создание и обучение модели с использованием GridSearchCV
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('regressor', RandomForestRegressor(random_state=42))])

param_grid = {
    'regressor__n_estimators': [50, 100, 200],
    'regressor__max_depth': [None, 10, 20, 30],
    'regressor__min_samples_split': [2, 5, 10]
}

grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Прогнозирование и оценка модели
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f'Best Model RMSE: {rmse}')

# Сохранение модели
joblib.dump(best_model, 'best_model.pkl')

# Сохранение порядка признаков
feature_order = features.columns.tolist()
joblib.dump(feature_order, 'feature_order.pkl')


Best Model RMSE: 1.6305013317686319


['feature_order.pkl']

#Код для Flask-приложения, которое использует обученную модель для предсказания уровня удовлетворенности на основе пользовательского ввода.

In [ ]:
app = Flask(__name__)

# Загрузка модели и порядка признаков
model = joblib.load('best_model.pkl')
feature_order = joblib.load('feature_order.pkl')

# HTML шаблоны
home_template = '''
<form action="/predict" method="post">
    Age: <input type="number" name="age" required><br>
    Gender:
    <select name="gender" required>
        <option value="Male">Male</option>
        <option value="Female">Female</option>
    </select><br>
    Health: <input type="number" name="health" min="1" max="5" required><br>
    Stress Level: <input type="number" name="stress_level" min="1" max="5" required><br>
    Income: <input type="number" name="income" required><br>
    Exercise Frequency: <input type="number" name="exercise_frequency" min="0" max="7" required><br>
    Sleep Hours: <input type="number" name="sleep_hours" min="4" max="10" step="0.1" required><br>
    Diet Quality: <input type="number" name="diet_quality" min="1" max="5" required><br>
    <input type="submit" value="Submit">
</form>
'''
# Здесь создается HTML-форма, которая позволяет пользователю вводить данные о себе. Форма отправляет данные на маршрут `/predict`.

# Шаблон результата
result_template = '''
<h2>Prediction Result</h2>
<p>Your predicted satisfaction level is: {{ prediction }}</p>
'''
 #Этот шаблон используется для отображения результата предсказания после отправки формы.

#Определение маршрутов

@app.route('/')
def home():
    return render_template_string(home_template)
#Этот маршрут обрабатывает GET-запросы на корневой URL (`/`). Он возвращает HTML-шаблон с формой для ввода данных.

#Маршрут предсказания
@app.route('/predict', methods=['POST'])
def predict():
    if request.method == 'POST':
        try:
            age = int(request.form['age'])
            gender = request.form['gender']
            health = int(request.form['health'])
            stress_level = int(request.form['stress_level'])
            income = int(request.form['income'])
            exercise_frequency = int(request.form['exercise_frequency'])
            sleep_hours = float(request.form['sleep_hours'])
            diet_quality = int(request.form['diet_quality'])

            data = {
                'age': [age],
                'gender_Male': [1 if gender == 'Male' else 0],
                'health': [health],
                'stress_level': [stress_level],
                'income': [income],
                'exercise_frequency': [exercise_frequency],
                'sleep_hours': [sleep_hours],
                'diet_quality': [diet_quality]
            }

            df = pd.DataFrame(data)
            df = df[feature_order]  # Убедимся, что порядок признаков соответствует

            prediction = model.predict(df)

            return render_template_string(result_template, prediction=prediction[0])
        except ValueError as e:
            return str(e)
#- Этот маршрут обрабатывает POST-запросы на `/predict`, когда пользователь отправляет форму.
#- Данные извлекаются из `request.form`, преобразуются в нужные типы (например, `int`, `float`) и помещаются в словарь `data`.
#- Создается DataFrame из этого словаря.
#- Порядок признаков в DataFrame приводится в соответствие с загруженным порядком (`feature_order`).
#- Модель делает предсказание на основе подготовленных данных.
#- Результат предсказания отображается с помощью шаблона `result_template`.

#Запуск приложения

if __name__ == '__main__':
    # Установить туннель ngrok
    public_url = ngrok.connect(addr="5000")
    print('Public URL:', public_url)

    # Запуск Flask приложения
    app.run(port=5000)


Public URL: NgrokTunnel: "https://518b-34-135-105-129.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [06/Aug/2024 16:20:49] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Aug/2024 16:20:49] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [06/Aug/2024 16:29:45] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Aug/2024 18:19:33] "GET / HTTP/1.1" 200 -


Проверяется, запускается ли скрипт как основная программа (`__name__ == '__main__'`). Если это так:
  - Создается туннель ngrok к локальному серверу на порту 5000, что позволяет получить публичный URL для доступа к приложению из Интернета.
  - Запускается сервер Flask на порту 5000.

1. Пользователь открывает главную страницу по публичному URL и видит форму для ввода данных.
2. После заполнения формы и нажатия кнопки "Submit", данные отправляются на сервер через POST-запрос.
3. Сервер обрабатывает запрос, извлекает данные и создает DataFrame с необходимыми признаками.
4. Модель делает предсказание уровня удовлетворенности на основе введенных данных.
5. Результат отображается пользователю на новой странице.


#Это приложение предоставляет простой интерфейс для взаимодействия с моделью машинного обучения и позволяет пользователям получать предсказания по введенным данным.